In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

### Generate synthetic data

In [2]:
from synthetics import generate_single_task_unipolar, gaussian_bags_of_words, vocab1k

N = 1000
M = 20
K = 2
NUM_SPLITS = 3

Ds = [] # data
Xs = [] # features
Ls = [] # noisy labels
Ys = [] # true labels
for _ in range(NUM_SPLITS):
    L, Y, _ = generate_single_task_unipolar(
        N, M, k=K, acc=[0.6, 0.9], rec=[0.1, 0.2], 
        class_balance=[0.3, 0.7], lf_balance=None, seed=1)
    
    X, D = gaussian_bags_of_words(Y, vocab1k)
    
    Ls.append(L)
    Ys.append(Y)
    Ds.append(D)
    Xs.append(X)

### Apply LabelModel

In [3]:
from metal.label_model import LabelModel

lm = LabelModel(seed=2)
lm.train(Ls[0], n_epochs=50)
Y_p = lm.score(Ls[1], Ys[1])

Overwriting seed=None to seed=2
Overwriting n_epochs=100 to n_epochs=50
[Epoch 0] Loss: 0.197513
[Epoch 10] Loss: 0.155587
[Epoch 20] Loss: 0.087589
[Epoch 30] Loss: 0.057624
[Epoch 40] Loss: 0.058253
[Epoch 49] Loss: 0.056262
Finished Training
Accuracy: 0.635


In [4]:
Y_p = lm.predict(Ls[1])

### Metrics

Calculate metrics in one of two ways.

1. Use metric_score() and pass the metric name
3. The the specific metric's function (e.g., accuracy_score())

In [5]:
from metal.metrics import metric_score, accuracy_score

metric_score(Ys[1], Y_p, 'accuracy')
accuracy_score(Ys[1], Y_p)

0.635

0.635

Built-in metrics include:

In [6]:
metric_score(Ys[1], Y_p, 'accuracy')
metric_score(Ys[1], Y_p, 'coverage')
metric_score(Ys[1], Y_p, 'precision')
metric_score(Ys[1], Y_p, 'recall')
metric_score(Ys[1], Y_p, 'f1')
metric_score(Ys[1], Y_p, 'fbeta', beta=1.0)

0.635

1.0

0.4461028192371476

0.8966666666666666

0.5957918050941307

0.5957918050941307

### Confusion matrix

In [10]:
from metal.analysis import confusion_matrix
confusion_matrix(Y_p, Ys[1])

array([[269, 334],
       [ 31, 366]])

In [11]:
confusion_matrix(Y_p, Ys[1], normalize=True)

array([[0.269, 0.334],
       [0.031, 0.366]])

In [12]:
mat = confusion_matrix(Y_p, Ys[1], pretty=True)

        y=1    y=2   
 l=1    269    334   
 l=2    31     366   


### Error analysis

In [20]:
from metal.analysis import error_buckets

buckets = error_buckets(Y_p, Ys[1], Ds[1])
tp = buckets[1,1]

In [21]:
tp[:5]

['and generation material artist thought recognize executive radio town analysis town cost but remain tax government relationship explain write name example able later reduce first price budget stuff majority source indeed large law spend east employee mrs less guess either address move of pressure window',
 'order organization eye writer wear various mrs drug appear hospital development create politics partner piece figure focus perform leader similar part head woman attack chair already far treatment will technology yeah',
 'accept like among outside fine receive kill condition customer not street when risk hotel leg yeah piece window risk other part suffer source majority wife leader free congress exist fail claim pay recognize fly finally',
 'provide realize wish voice artist catch election tonight month address story heavy heavy vote ask your themselves around few receive recognize state upon nor oil guess hard fight agency floor fall individual suffer husband stand win stay relig